Trabajo final 
Redees Neuronales
Flores Lara Alberto 6BV1

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Input, Add, SeparableConv2D, Resizing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score, f1_score
import time
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50

In [19]:
# Configuración de parámetros
NUM_CLASES = 10
EPOCHS = 30
BATCH_SIZE = 64
IMAGE_SIZE = (224, 224)  # Tamaño requerido por VGG16 y ResNet50

# 1. Cargar y Preprocesar CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalizar los valores de los píxeles en el rango [0, 1]
x_train_resized = x_train.astype('float32') / 255.0
x_test_resized = x_test.astype('float32') / 255.0

# Convertir las etiquetas a one-hot encoding
y_train_cat = to_categorical(y_train, NUM_CLASES)
y_test_cat = to_categorical(y_test, NUM_CLASES)

# Lista para almacenar los resultados de cada modelo
results = []

In [20]:
# 2. Definir Funciones para Construir Modelos

# 2.1. Función para construir MLP
def build_mlp(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(NUM_CLASES, activation='softmax'))
    return model

# 2.2. Función para construir CNN básica
def build_cnn(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(NUM_CLASES, activation='softmax'))
    return model

# 2.3. Función para construir CNN con regularización
def build_cnn_regularized(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASES, activation='softmax'))
    return model

# 2.4. Función para construir CNN avanzada
def build_cnn_advanced(input_shape):
    input_layer = Input(shape=input_shape)

    # Primera capa convolucional con Batch Normalization
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Segunda capa convolucional separable
    x = SeparableConv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    # Residual block
    residual = Conv2D(64, (1, 1), padding='same')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Add()([x, residual])  # Conexión residual

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output_layer = Dense(NUM_CLASES, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# 2.5. Función para construir Transfer Learning (ResNet50))
def build_fine_tuning_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Congelar todas las capas inicialmente
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(Resizing(224, 224, input_shape=input_shape))
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# 2.6. Función para construir Fine Tuning (ResNet50)
def fine_tune_resnet50(model, base_model, x_train, y_train, x_test, y_test, epochs=10, batch_size=32):
    # Descongelar las últimas 4 capas del modelo base
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    # Compilar el modelo con una tasa de aprendizaje más baja
    optimizer = Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, y_test),
        verbose=2
    )
    end_time = time.time()

    # Predicciones
    y_pred = np.argmax(model.predict(x_test), axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Cálculo de métricas
    accuracy = np.mean(y_pred == y_true)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Almacenar resultados
    results.append({
        'Modelo': 'Fine Tuning ResNet50 (Final)',
        'Exactitud': accuracy,
        'Precisión': precision,
        'Recall': recall,
        'F1-score': f1,
        'Tiempo de Entrenamiento (s)': end_time - start_time
    })

    print(f"Modelo: Fine Tuning ResNet50 (Final)")
    print(f"Exactitud: {accuracy:.4f}, Precisión: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos\n")

    return history, end_time - start_time


In [21]:
def train_and_evaluate(model, model_name, x_train, y_train, x_test, y_test, epochs=EPOCHS, batch_size=BATCH_SIZE, fine_tune=False):
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, y_test),
        verbose=2
    )
    end_time = time.time()

    # Predicciones
    y_pred = np.argmax(model.predict(x_test), axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Cálculo de métricas
    accuracy = np.mean(y_pred == y_true)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Almacenar resultados
    results.append({
        'Modelo': model_name,
        'Exactitud': accuracy,
        'Precisión': precision,
        'Recall': recall,
        'F1-score': f1,
        'Tiempo de Entrenamiento (s)': end_time - start_time
    })

    print(f"Modelo: {model_name}")
    print(f"Exactitud: {accuracy:.4f}, Precisión: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos\n")

    return history, end_time - start_time


In [ ]:
# 4.1. Entrenar y evaluar MLP
print("Entrenando MLP...")
model_mlp = build_mlp((32, 32, 3))  # Input_shape original
history_mlp, time_mlp = train_and_evaluate(model_mlp, 'MLP', x_train_resized, y_train_cat, x_test_resized, y_test_cat)

# 4.2. Entrenar y evaluar CNN básica
print("Entrenando CNN Básica...")
model_cnn = build_cnn((32, 32, 3))  # Input_shape original
history_cnn, time_cnn = train_and_evaluate(model_cnn, 'CNN Básica', x_train_resized, y_train_cat, x_test_resized, y_test_cat)

# 4.3. Entrenar y evaluar CNN regularizada
print("Entrenando CNN Regularizada...")
model_cnn_reg = build_cnn_regularized((32, 32, 3))  # Input_shape original
history_cnn_reg, time_cnn_reg = train_and_evaluate(model_cnn_reg, 'CNN Regularizada', x_train_resized, y_train_cat, x_test_resized, y_test_cat)

# 4.4. Entrenar y evaluar CNN avanzada
print("Entrenando CNN Avanzada...")
model_cnn_adv = build_cnn_advanced((32, 32, 3))  # Input_shape original
history_cnn_adv, time_cnn_adv = train_and_evaluate(model_cnn_adv, 'CNN Avanzada', x_train_resized, y_train_cat, x_test_resized, y_test_cat)
# 5. Entrenar y Evaluar Modelos de Transfer Learning y Fine Tuning

# 5.1. Transfer Learning con ResNet50
print("Entrenando Transfer Learning con ResNet50...")
model_resnet50 = build_fine_tuning_resnet50((32, 32, 3), NUM_CLASES)
history_resnet50, time_resnet50 = train_and_evaluate(model_resnet50, 'Fine Tuning ResNet50 (Inicial)', x_train_resized, y_train_cat, x_test_resized, y_test_cat)

# 5.2. Fine Tuning con ResNet50 (Entrenamiento Inicial con Capas Congeladas)
print("Entrenando Fine Tuning con ResNet50 (Inicial)...")
# Extraer el modelo base de ResNet50 para Fine Tuning
base_model_resnet50 = model_resnet50.layers[0]

# Realizar Fine Tuning (Descongelar últimas 4 capas y reentrenar)
history_resnet50_finetune, time_resnet50_finetune = fine_tune_resnet50(
    model_resnet50, 
    base_model_resnet50, 
    x_train_resized, 
    y_train_cat, 
    x_test_resized, 
    y_test_cat
)


In [ ]:
# Crear DataFrame con los resultados
df_results = pd.DataFrame(results)
print(df_results)

# Gráficas de las métricas
metrics = ['Exactitud', 'Precisión', 'Recall', 'F1-score', 'Tiempo de Entrenamiento (s)']
num_metrics = len(metrics)

plt.figure(figsize=(20, 15))

for i, metric in enumerate(metrics, 1):
    plt.subplot(3, 2, i)
    plt.bar(df_results['Modelo'], df_results[metric], color='skyblue')
    plt.title(metric)
    plt.ylabel(metric)
    plt.xticks(rotation=45, ha='right')
    for index, value in enumerate(df_results[metric]):
        if metric != 'Tiempo de Entrenamiento (s)':
            plt.text(index, value + 0.005, f"{value:.2f}", ha='center', va='bottom')
        else:
            plt.text(index, value + 1, f"{value:.2f}", ha='center', va='bottom')

plt.tight_layout()
plt.show()
